In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import (
    UnstructuredFileLoader,
)  # 이건 더이상 사용하지 말라고 하나 langchain-unstructured 충돌하고 있기 때문에 임시로 쓴다
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_chroma import Chroma
from langchain_classic.storage import LocalFileStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [ ]:
# basic LAG
llm = ChatOpenAI(temperature=0.1)

splitter = RecursiveCharacterTextSplitter(
    chunk_overlap=50,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    {
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    }
)

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Victory Mansions")

C:\Users\myjis\AppData\Local\Temp\ipykernel_13264\3445031986.py:7: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-unstructured package and should be used instead. To use it run `pip install -U `langchain-unstructured` and import as `from `langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader("./files/chapter_one.txt")
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
c:\Users\myjis\myFiles\FullStackGPT\.venv\Lib\site-packages\langchain_classic\embeddings\cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE

AIMessage(content='Victory Mansions is a building where Winston Smith lives in the novel "1984" by George Orwell. It is described as a run-down apartment complex located in London, chief city of Airstrip One. The building is dilapidated, with rotting nineteenth-century houses, patched windows, and sagging garden walls. The Ministry of Truth, where Winston works, towers above the landscape and is one of the four Ministries that dominate the city. Victory Mansions is a grim and oppressive place, reflecting the bleak and totalitarian society depicted in the novel.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 3534, 'total_tokens': 3647, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerpri

In [4]:
splitter = CharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1741, which is longer than the specified 600
Created a chunk of size 2001, which is longer than the specified 600
Created a chunk of size 1900, which is longe

In [14]:
retriever.invoke("Describe Victory Mansions")

[Document(id='d841172a-25fb-4efd-b5c2-b2d145f9a821', metadata={'source': './files/chapter_one.txt'}, page_content="Winston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast. He took down from the shelf a bottle of colourless liquid with a plain white label marked VICTORY GIN. It gave off a sickly, oily smell, as of Chinese ricespirit. Winston poured out nearly a teacupful, nerved himself for a shock, and gulped it down like a dose of medicine."),
 Document(id='f71f962d-f13c-4e6e-bd88-5cfc82ff6831', metadata={'source': './files/chapter_one.txt'}, page_content="Winston turned round abruptly. He had set his features into the ex

In [ ]:
# Map Rduce LCEL Chain
llm = ChatOpenAI(temperature=0.1)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. 
            Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)


map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(map_doc_chain.invoke({
        "context": doc.page_content,
        "question": question
    }).content for doc in documents)

    results = []
    for documnet in documents:
        result = map_doc_chain.invoke(
            {"context": documnet.page_content, "question": question}
        ).content
        results.append(result)
    return "\n\n".join(results)


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
        Given the following extracted parts of a long documnet and qestion,
        create a final answer.
        If you don't know the answer, just say that you don't know. Don't try to make up an answer
        _____
        {context}
    """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("Describe Victory Mansions")

질문에 대한 답
Victory Mansions is a building complex where Winston resides in the novel "1984" by George Orwell. It is described as a dilapidated apartment building with run-down facilities and poor living conditions. The building is located in Airstrip One, a province of Oceania, and is characterized by its grim and oppressive atmosphere. Winston's apartment in Victory Mansions is small and cramped, with a telescreen that constantly monitors his activities. The building symbolizes the bleak and oppressive nature of the society depicted in the novel.
질문에 대한 답
Victory Mansions is a dilapidated apartment building where Winston Smith, the protagonist of George Orwell's novel "1984," resides. The building is described as run-down, with shabby walls and a faulty elevator. The apartments are cramped and poorly maintained, reflecting the oppressive and impoverished conditions of the society in which Winston lives.
질문에 대한 답
Victory Mansions is a dilapidated apartment building where Winston Smith, t

AIMessage(content='Victory Mansions is a dilapidated apartment building where Winston Smith, the protagonist of George Orwell\'s novel "1984," resides. The building is described as run-down, with shabby walls and broken elevators. The apartments are cramped and poorly maintained, reflecting the oppressive and impoverished conditions of the society in which Winston lives.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 395, 'total_tokens': 462, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CegwJ1XcLpkZa6wBNSqYdbmcykUDI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--fc584059-cca6-4ad7-bd9b-327f54052b79-0', usage_metadata